<a href="https://colab.research.google.com/github/mahfuz978/DS-Unit-2-Regression-Classification/blob/master/module2/Mahfuzur's_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module2')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 19.30 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (71/71), done.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 112kB 36.2MB/s 
     |████████████████████████████████| 133kB 50.1MB/s 
     |████████████████████████████████| 57.7MB 374kB/s 
     |████████████████████████████████| 235kB 46.8MB/s 
     |████████████████████████████████| 614kB 43.4MB/s 
     |████████████████████████████████| 235kB 54.3MB/s 
     |████████████████████████████████| 307kB 47.8MB/s 
     |████████████████████████████████| 215kB 47.4MB/s 
     |████████████████████████████████| 51kB 13.6MB/s 
     |█████████

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
df.head(3)

bathrooms  bedrooms  ... wheelchair_access common_outdoor_space
0        1.5         3  ...                 0                    0
1        1.0         2  ...                 0                    0
2        1.0         1  ...                 0                    0

[3 rows x 34 columns]

In [0]:
df['created'] = pd.to_datetime(df['created'])

In [0]:
df.describe()

bathrooms      bedrooms  ...  wheelchair_access  common_outdoor_space
count  48817.000000  48817.000000  ...       48817.000000          48817.000000
mean       1.201794      1.537149  ...           0.027224              0.026241
std        0.470711      1.106087  ...           0.162738              0.159852
min        0.000000      0.000000  ...           0.000000              0.000000
25%        1.000000      1.000000  ...           0.000000              0.000000
50%        1.000000      1.000000  ...           0.000000              0.000000
75%        1.000000      2.000000  ...           0.000000              0.000000
max       10.000000      8.000000  ...           1.000000              1.000000

[8 rows x 29 columns]

In [0]:
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                  object
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [0]:
df.head()

bathrooms  bedrooms  ... wheelchair_access common_outdoor_space
0        1.5         3  ...                 0                    0
1        1.0         2  ...                 0                    0
2        1.0         1  ...                 0                    0
3        1.0         1  ...                 0                    0
4        1.0         4  ...                 0                    0

[5 rows x 34 columns]

In [0]:
st_apr = df.loc[df['created']>='4/1/2016',:]
train = st_apr.loc[st_apr['created']<='5/31/2016',:]
test = df.loc[df['created']>='6/1/2016',:]
print(train.head());
print(test.head());


   bathrooms  bedrooms  ... wheelchair_access common_outdoor_space
2        1.0         1  ...                 0                    0
3        1.0         1  ...                 0                    0
4        1.0         4  ...                 0                    0
5        2.0         4  ...                 0                    0
6        1.0         2  ...                 0                    0

[5 rows x 34 columns]
    bathrooms  bedrooms  ... wheelchair_access common_outdoor_space
0         1.5         3  ...                 0                    0
1         1.0         2  ...                 0                    0
11        1.0         1  ...                 0                    0
14        1.0         1  ...                 0                    0
24        2.0         4  ...                 0                    0

[5 rows x 34 columns]


In [0]:
import plotly.express as px

In [0]:
train['bedrooms'].mean()

1.5282515154370426

In [0]:
test['bedrooms'].mean()

1.5536440228598363

In [0]:
from sklearn.metrics import mean_absolute_error

target = 'bedrooms'
y_train = train[target]
y_test = test[target]
baseline = y_train.mean()
y_pred = [baseline] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error for bedrooms from April and May is :{mae:.2f}')

y_pred = [baseline] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error for bedrooms from June is:{mae:.2f} ')

Train Error for bedrooms from April and May is :0.93
Test Error for bedrooms from June is:0.93 


In [0]:
px.scatter(
    train,
    x='bedrooms',
    y='price',
    title='Price vs. number of bedrooms, data from April and May 2016 ',
    trendline='ols',)

In [0]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
features = ['bedrooms']
X_train = train[features]
X_test = test[features]
print(f'Linear Regression, dependent on: {features}')

model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: {mae:.2f} percent')


y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: {mae1:.2f} percent')

Linear Regression, dependent on: ['bedrooms']
Train Error: 0.00 percent
Test Error: 0.00 percent


In [0]:
px.scatter_3d(
    train,
    x='bedrooms', 
    y='bathrooms', 
    z='price', 
    title='Price vs. number of bedrooms & bathrooms, data from April and May 2016'
)

In [0]:
features = ['bedrooms', 
            'bathrooms']
print(f'Linear Regression, dependent on: {features}')
X_train = train[features]
X_test = test[features]

model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_train_pred)
print('The mean absolute error, train:',train_mae)

test_mae = mean_absolute_error(y_test, model.predict(X_test))
print('The mean absolute error, test :',test_mae)

Linear Regression, dependent on: ['bedrooms', 'bathrooms']
The mean absolute error, train: 1.4475685670292524e-15
The mean absolute error, test : 1.4420494649273127e-15


In [0]:
import itertools
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

def regression_3d(df, x, y, z, num=100, **kwargs):
    """
    Visualize linear regression in 3D: 2 features + 1 target
    
    df : Pandas DataFrame
    x : string, feature 1 column in df
    y : string, feature 2 column in df
    z : string, target column in df
    num : integer, number of quantiles for each feature
    """
    
    # Plot data
    fig = px.scatter_3d(df, x, y, z, **kwargs)
    
    # Fit Linear Regression
    features = [x, y]
    target = z
    model = LinearRegression()
    model.fit(df[features], df[target])    
    
    # Define grid of coordinates in the feature space
    xmin, xmax = df[x].min(), df[x].max()
    ymin, ymax = df[y].min(), df[y].max()
    xcoords = np.linspace(xmin, xmax, num)
    ycoords = np.linspace(ymin, ymax, num)
    coords = list(itertools.product(xcoords, ycoords))
    
    # Make predictions for the grid
    predictions = model.predict(coords)
    Z = predictions.reshape(num, num).T
    
    # Plot predictions as a 3D surface (plane)
    fig.add_trace(go.Surface(x=xcoords, y=ycoords, z=Z))
    
    return fig

In [0]:
regression_3d(
    train,
    x='bedrooms', 
    y='bathrooms', 
    z='price', 
    title='Price vs. number of bedrooms & bathrooms, data from April and May 2016'
)

In [0]:
model.intercept_, model.coef_

(-2.4424906541753444e-15, array([1.]))

In [0]:
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept -2.4424906541753444e-15
bedrooms    1.0
